# JARVIS Baseline Model w/Featurizer

see all datasets here: https://pages.nist.gov/jarvis/databases/

In [ ]:
# Imports
import io
import sys
import toml
import pprint

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from jarvis.db.figshare import data


import matplotlib
%matplotlib inline
import matplotlib.pyplot as plt
from matplotlib.gridspec import GridSpec

import importlib

from sklearn.model_selection import train_test_split
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.preprocessing import OneHotEncoder

## Load TOML Configuration

In [ ]:
# Configuration
CONFIG_PATH = "config.toml"
config = toml.load(CONFIG_PATH)

# Pretty print configuration
print("Project Configuration:")
pprint.pprint(config)

# Set up system path
SYS_PATH = config.get('system', {}).get('path', './')
sys.path.append(SYS_PATH) # .../code/jarvis/


Project Configuration:
{'data': {'dataset_name': 'dft_3d', 'store_dir': '/shared/data/jarvis'},
 'features': {'bag_of_elements': True,
              'derived': ['eps_mean', 'eps_std'],
              'use_columns': ['ehull',
                              'formation_energy_peratom',
                              'avg_elec_mass',
                              'avg_hole_mass',
                              'effective_masses_297K',
                              'epsx',
                              'epsy',
                              'epsz',
                              'natoms']},
 'filters': {'bandgap_column': 'optb88vdw_bandgap',
             'max_eps': 10.0,
             'min_eps': 1.0,
             'semiconductor_max': 4.0,
             'semiconductor_min': 0.5,
             'toxic_elements': ['Pb', 'Cd', 'Hg', 'As', 'Se'],
             'transparent_min': 3.0},
 'known': {'transparent_formulas': ['In2O3',
                                    'ZnO',
                                   

In [ ]:
# Custom Imports and Configurations
from jarvis_utils import load_or_fetch_dataset
from logger_utils import setup_logger, flush_logger
from filter_utils import apply_filters

# Setup logger
logger = setup_logger(config)

logger.info("Project configuration loaded.")
logger.info(f"Dataset: {config['data']['dataset_name']}")
logger.info(f"Store directory: {config['data']['store_dir']}")

# Load dataset
df = load_or_fetch_dataset(config["data"]["dataset_name"], data, config["data"]["store_dir"])
logger.info(f"Dataset shape: {df.shape}")


2025-11-25 20:16:22,988 - jarvis_project - INFO - Project configuration loaded.
2025-11-25 20:16:22,989 - jarvis_project - INFO - Dataset: dft_3d
2025-11-25 20:16:22,989 - jarvis_project - INFO - Store directory: /shared/data/jarvis


Loading existing pickle file: /shared/data/jarvis/jarvis_dft_3d.pkl


2025-11-25 20:16:24,620 - jarvis_project - INFO - Dataset shape: (75993, 64)


Dataset shape: (75993, 64)


In [ ]:
features = df.columns.tolist()
logger.info(f"Features: {features}")

2025-11-25 20:16:24,628 - jarvis_project - INFO - Features: ['jid', 'spg_number', 'spg_symbol', 'formula', 'formation_energy_peratom', 'func', 'optb88vdw_bandgap', 'atoms', 'slme', 'magmom_oszicar', 'spillage', 'elastic_tensor', 'effective_masses_300K', 'kpoint_length_unit', 'maxdiff_mesh', 'maxdiff_bz', 'encut', 'optb88vdw_total_energy', 'epsx', 'epsy', 'epsz', 'mepsx', 'mepsy', 'mepsz', 'modes', 'magmom_outcar', 'max_efg', 'avg_elec_mass', 'avg_hole_mass', 'icsd', 'dfpt_piezo_max_eij', 'dfpt_piezo_max_dij', 'dfpt_piezo_max_dielectric', 'dfpt_piezo_max_dielectric_electronic', 'dfpt_piezo_max_dielectric_ionic', 'max_ir_mode', 'min_ir_mode', 'n-Seebeck', 'p-Seebeck', 'n-powerfact', 'p-powerfact', 'ncond', 'pcond', 'nkappa', 'pkappa', 'ehull', 'Tc_supercon', 'dimensionality', 'efg', 'xml_data_link', 'typ', 'exfoliation_energy', 'spg', 'crys', 'density', 'poisson', 'raw_files', 'nat', 'bulk_modulus_kv', 'shear_modulus_gv', 'mbj_bandgap', 'hse_gap', 'reference', 'search']


## Explore numberic and categorical features, add descriptions for each.

In [ ]:

# --- Feature audit utility ---
def audit_features(df: pd.DataFrame, descriptions: dict = None, n_examples: int = 1) -> pd.DataFrame:
    """
    Inspect dataframe columns, separate numeric vs categorical, and show example values + descriptions.
    
    Parameters
    ----------
    df : pd.DataFrame
        Original dataset.
    descriptions : dict
        Optional mapping {column_name: description}.
    n_examples : int
        Number of example values to show per column.
    
    Returns
    -------
    pd.DataFrame
        Summary table of features.
    """
    records = []
    for col in df.columns:
        dtype = df[col].dtype
        # get example values (first non-null)
        examples = df[col].dropna().head(n_examples).tolist()
        example_str = ", ".join(map(str, examples)) if examples else "NaN"
        desc = descriptions.get(col, "No description available") if descriptions else "No description available"
        records.append({
            "Feature": col,
            "Type": str(dtype),
            "Example": example_str,
            "Description": desc
        })
    return pd.DataFrame(records)

# Load descriptions
config = toml.load("features.toml")
feature_descriptions = config["features"]

audit_df = audit_features(df, descriptions=feature_descriptions, n_examples=2)

# Apply to audit table
audit_df["Description"] = audit_df["Feature"].apply(
    lambda f: feature_descriptions.get(f, "No description available")
)

pd.set_option("display.max_rows", None)   # show all rows
pd.set_option("display.max_columns", None)  # show all columns
pd.set_option("display.width", None)     # don't wrap columns
pd.set_option("display.max_colwidth", None)  # show full cell contents

display(audit_df)


,Feature,Type,Example,Description
0,jid,object,"JVASP-90856, JVASP-86097",Unique JARVIS material identifier
1,spg_number,object,"129, 221","Space group number (crystallographic symmetry, integer)"
2,spg_symbol,object,"P4/nmm, Pm-3m","Space group symbol (e.g., P4/nmm, Cmmm)"
3,formula,object,"TiCuSiAs, DyB6",Chemical formula of the material
4,formation_energy_peratom,float64,"-0.42762, -0.41596",Formation energy per atom (eV/atom)
5,func,object,"OptB88vdW, OptB88vdW","Exchange-correlation functional used in calculation (e.g., OptB88vdW)"
6,optb88vdw_bandgap,float64,"0.0, 0.0",Bandgap computed with optB88vdW functional (eV)
7,atoms,object,"{'lattice_mat': [[3.566933224304235, 0.0, -0.0], [0.0, 3.566933224304235, -0.0], [-0.0, -0.0, 9.397075454186664]], 'coords': [[2.6751975000000003, 2.6751975000000003, 7.376101754328542], [0.8917325, 0.8917325, 2.0209782456714573], [0.8917325, 2.6751975000000003, 4.69854], [2.6751975000000003, 0.8917325, 4.69854], [0.8917325, 2.6751975000000003, 0.0], [2.6751975000000003, 0.8917325, 0.0], [2.6751975000000003, 2.6751975000000003, 2.8894795605846353], [0.8917325, 0.8917325, 6.507600439415366]], 'elements': ['Ti', 'Ti', 'Cu', 'Cu', 'Si', 'Si', 'As', 'As'], 'abc': [3.56693, 3.56693, 9.39708], 'angles': [90.0, 90.0, 90.0], 'cartesian': True, 'props': ['', '', '', '', '', '', '', '']}, {'lattice_mat': [[4.089078911208881, 0.0, 0.0], [-0.0, 4.089078911208881, -0.0], [0.0, -0.0, 4.089078911208881]], 'coords': [[0.0, 0.0, 0.0], [0.8121488741123271, 2.04454, 2.04454], [2.04454, 3.276931125887674, 2.04454], [2.04454, 0.8121488741123271, 2.04454], [3.276931125887674, 2.04454, 2.04454], [2.04454, 2.04454, 0.8121488741123271], [2.04454, 2.04454, 3.276931125887674]], 'elements': ['Dy', 'B', 'B', 'B', 'B', 'B', 'B'], 'abc': [4.08908, 4.08908, 4.08908], 'angles': [90.0, 90.0, 90.0], 'cartesian': True, 'props': ['', '', '', '', '', '', '']}","Full atomic structure: lattice, coordinates, elements, angles"
8,slme,object,"na, na",Spectroscopic limited maximum efficiency (solar cell metric)
9,magmom_oszicar,object,"0.0, 0.0",Magnetic moment from OSZICAR file (µB)


### Featurization Plan


| Feature                     | Plan              | Notes                                                                 |
|-----------------------------|-------------------|-----------------------------------------------------------------------|
| jid                         | Drop              | Identifier only                                                       |
| spg_number                  | Numeric           | Convert to int; crystallographic symmetry                             |
| spg_symbol                  | Categorical       | One‑hot/frequency encode                                              |
| formula                     | Flatten           | Bag‑of‑elements indicators, composition vectors                       |
| formation_energy_peratom    | Numeric           | Keep as float                                                         |
| func                        | Categorical       | Encode functional used                                                |
| optb88vdw_bandgap           | Numeric           | Keep as float                                                         |
| atoms                       | Network structure | Use atomic graph featurizer (lattice, coords, elements)               |
| slme                        | Numeric           | Convert to float if available                                         |
| magmom_oszicar              | Numeric           | Magnetic moment scalar                                                |
| spillage                    | Numeric           | Topological indicator                                                 |
| elastic_tensor              | Flatten           | Extract Voigt averages, shear/bulk moduli                             |
| effective_masses_300K       | Flatten           | Extract electron/hole masses                                          |
| kpoint_length_unit          | Numeric           | Convert to int                                                        |
| maxdiff_mesh                | Drop              | Convergence diagnostic                                                |
| maxdiff_bz                  | Drop              | Convergence diagnostic                                                |
| encut                       | Numeric           | Plane‑wave cutoff energy                                              |
| optb88vdw_total_energy      | Numeric           | Keep as float                                                         |
| epsx, epsy, epsz            | Numeric + Combine | Compute mean/std; keep individual                                     |
| mepsx, mepsy, mepsz         | Numeric           | If available                                                          |
| modes                       | Flatten           | Extract mean/std/min/max of phonon modes                              |
| magmom_outcar               | Numeric           | Magnetic moment scalar                                                |
| max_efg                     | Numeric           | Electric field gradient                                               |
| avg_elec_mass, avg_hole_mass| Numeric           | Effective masses                                                      |
| icsd                        | Drop              | Identifier                                                            |
| dfpt_piezo_max_eij, dij     | Numeric           | Piezoelectric coefficients                                            |
| dfpt_piezo_max_dielectric   | Numeric           | Dielectric constant                                                   |
| dfpt_piezo_max_dielectric_electronic, ionic | Numeric | Contributions                                                         |
| max_ir_mode, min_ir_mode    | Numeric           | Phonon frequencies                                                    |
| n-Seebeck, p-Seebeck        | Numeric           | Seebeck coefficients                                                  |
| n-powerfact, p-powerfact    | Numeric           | Power factors                                                         |
| ncond, pcond                | Numeric           | Conductivities                                                        |
| nkappa, pkappa              | Numeric           | Thermal conductivities                                                |
| ehull                       | Numeric           | Stability metric                                                      |
| Tc_supercon                 | Numeric           | Superconducting Tc                                                    |
| dimensionality              | Categorical       | Encode (bulk, layered, etc.)                                          |
| efg                         | Flatten           | Tensor → scalar summaries                                             |
| xml_data_link               | Drop              | Identifier                                                            |
| typ                         | Categorical       | Encode (bulk, monolayer, etc.)                                        |
| exfoliation_energy          | Numeric           | Convert to float                                                      |
| spg                         | Drop              | Duplicate of spg_number                                               |
| crys                        | Categorical       | Encode crystal system                                                 |
| density                     | Numeric           | Keep as float                                                         |
| poisson                     | Numeric           | Convert to float                                                      |
| raw_files                   | Drop              | Structural artifact                                                   |
| nat                         | Numeric           | Number of atoms                                                       |
| bulk_modulus_kv, shear_modulus_gv | Numeric     | Mechanical properties                                                 |
| mbj_bandgap, hse_gap        | Numeric           | Bandgaps from other functionals                                       |
| reference                   | Drop              | External ID                                                           |
| search                      | Drop              | Search string artifact                                                |


#### Summary
 - Drop: jid, icsd, xml_data_link, reference, search, raw_files, convergence diagnostics (maxdiff_mesh, maxdiff_bz), duplicate spg.
 - Numeric (keep/convert): formation_energy_peratom, bandgaps, energies, dielectric constants, mechanical moduli, density, ehull, Tc_supercon, Seebeck, conductivities, etc.
- Categorical (encode): spg_symbol, func, dimensionality, typ, crys.
- Flatten/aggregate: atoms (-> graph), elastic_tensor, effective_masses, phonon modes, efg.
- Network structure: atoms (lattice, coords, elements) -> graph neural network input.
- Combine: dielectric constants (epsx/epsy/epsz -> mean/std), phonon modes (mean/std/min/max).

## Test Featurizer

In [ ]:
from featurizer import Featurizer

# Initialize featurizer with TOML plan
featurizer = Featurizer("features_plan.toml")

# Apply plan to dataframe
df_transformed = featurizer.apply_plan(df)

print(df_transformed.head())


/home/brent/src/ml-final-project-team2/code/jarvis/featurizer.py:13: FutureWarning:

Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`



   spg_number spg_symbol  formation_energy_peratom       func  \
0         129     P4/nmm                  -0.42762  OptB88vdW   
1         221      Pm-3m                  -0.41596  OptB88vdW   
2         119      I-4m2                   0.04847  OptB88vdW   
3          14     P2_1/c                  -0.44140  OptB88vdW   
4         164      P-3m1                  -0.71026  OptB88vdW   

   optb88vdw_bandgap  slme  magmom_oszicar  spillage  kpoint_length_unit  \
0              0.000   NaN             0.0       NaN                60.0   
1              0.000   NaN             0.0       NaN                65.0   
2              0.000   NaN             0.0       NaN                60.0   
3              0.472   NaN             0.0       NaN                40.0   
4              0.000   NaN             0.0       NaN                40.0   

    encut  optb88vdw_total_energy      epsx      epsy      epsz   mepsx  \
0   650.0                -3.37474   76.2300   76.2300   54.0402     NaN   
1 

In [ ]:
df_transformed.head()

,spg_number,spg_symbol,formation_energy_peratom,func,optb88vdw_bandgap,slme,magmom_oszicar,spillage,kpoint_length_unit,encut,optb88vdw_total_energy,epsx,epsy,epsz,mepsx,mepsy,mepsz,magmom_outcar,max_efg,avg_elec_mass,avg_hole_mass,dfpt_piezo_max_eij,dfpt_piezo_max_dij,dfpt_piezo_max_dielectric,dfpt_piezo_max_dielectric_electronic,dfpt_piezo_max_dielectric_ionic,max_ir_mode,min_ir_mode,n-Seebeck,p-Seebeck,n-powerfact,p-powerfact,ncond,pcond,nkappa,pkappa,ehull,Tc_supercon,dimensionality,typ,exfoliation_energy,crys,density,poisson,nat,bulk_modulus_kv,shear_modulus_gv,mbj_bandgap,hse_gap,eps_mean,eps_std,modes_mean,modes_std,modes_min,modes_max
0,129,P4/nmm,-0.42762,OptB88vdW,0.000,NaN,0.0,NaN,60.0,650.0,-3.37474,76.2300,76.2300,54.0402,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0423,NaN,3D-bulk,bulk,NaN,tetragonal,5.956,NaN,8,NaN,NaN,NaN,NaN,68.833400,12.811287,NaN,NaN,NaN,NaN
1,221,Pm-3m,-0.41596,OptB88vdW,0.000,NaN,0.0,NaN,65.0,1000.0,-5.79186,140.9742,140.9742,140.9742,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0456,NaN,3D-bulk,bulk,NaN,cubic,5.522,NaN,7,NaN,NaN,NaN,NaN,140.974200,0.000000,NaN,NaN,NaN,NaN
2,119,I-4m2,0.04847,OptB88vdW,0.000,NaN,0.0,NaN,60.0,1100.0,-4.65924,215.4488,215.4488,159.4368,NaN,NaN,NaN,0.0,23.940,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.3183,NaN,intercalated ion,bulk,NaN,tetragonal,10.960,NaN,4,NaN,NaN,NaN,NaN,196.778133,32.338543,NaN,NaN,NaN,NaN
3,14,P2_1/c,-0.44140,OptB88vdW,0.472,NaN,0.0,NaN,40.0,500.0,-0.42496,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0000,NaN,intercalated ion,bulk,NaN,monoclinic,5.145,NaN,32,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,164,P-3m1,-0.71026,OptB88vdW,0.000,NaN,0.0,NaN,40.0,500.0,-3.87823,47.0290,47.0290,28.1625,79.071,78.7247,18.006,0.0,89.678,0.0,0.0,0.0,0.0,54.202007,44.924312,9.277695,342.84,204.14,18.273333,17.486667,657.17,614.083333,2.033191e+06,2.045951e+06,2.798133e+15,2.811808e+15,0.0156,NaN,2D-bulk,bulk,83.78,trigonal,5.718,0.23,3,48.79,33.05,0.0,NaN,40.740167,10.892579,135.216667,109.434119,-0.22,335.68


## Baseline Model using Featurizer

In [ ]:
# --- Imports ---
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.impute import SimpleImputer
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier

# --- Apply Featurizer ---
from featurizer import Featurizer
df_feat = df

# --- Apply Filters ---
from filter_utils import apply_filters


# Load TOML plan and apply
featurizer = Featurizer("features_plan.toml")
df_feat = featurizer.apply_plan(df_feat)


/home/brent/src/ml-final-project-team2/code/jarvis/featurizer.py:13: FutureWarning:

Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`



## Hack Hack

In [35]:
# Detect inf
df_feat = df_feat.drop(columns=["formula"], errors="ignore")
mask_inf = np.isinf(df_feat.select_dtypes(include=[np.number]))
if mask_inf.any().any():
    bad_cols = mask_inf.any()[mask_inf.any()].index.tolist()
    print("Columns with inf values:", bad_cols)


In [36]:
# Transparency window: 2.5-4.0 eV
bandgap_col = "optb88vdw_bandgap"
df_feat[bandgap_col] = pd.to_numeric(df_feat[bandgap_col], errors="coerce")

# Stability filter: ehull < 0.1 eV/atom
df_feat["ehull"] = pd.to_numeric(df_feat["ehull"], errors="coerce")

# Derived label: candidate transparent semiconductor
df_feat["is_candidate"] = (
    (df_feat[bandgap_col].between(2.5, 4.0)) &
    (df_feat["ehull"] < 0.1)
).astype(int)


In [37]:
X = df_feat.drop(columns=["is_candidate"], errors="ignore")
y = df_feat["is_candidate"]

X_train, X_val, y_train, y_val = train_test_split(
    X, y, test_size=0.2, stratify=y, random_state=42
)

numeric_cols = X_train.select_dtypes(include=[np.number]).columns.tolist()
cat_cols = X_train.select_dtypes(include=["object", "category"]).columns.tolist()

numeric = Pipeline(steps=[
    ("impute", SimpleImputer(strategy="median")),
    ("scale", StandardScaler())
])
categorical = Pipeline(steps=[
    ("impute", SimpleImputer(strategy="most_frequent")),
    ("encode", OneHotEncoder(handle_unknown="ignore", sparse_output=False))
])

preproc = ColumnTransformer(
    transformers=[
        ("num", numeric, numeric_cols),
        ("cat", categorical, cat_cols)
    ],
    remainder="drop"
)

clf = RandomForestClassifier(n_estimators=300, class_weight="balanced", random_state=42)
pipe = Pipeline(steps=[("preproc", preproc), ("clf", clf)])

pipe.fit(X_train, y_train)
val_score = pipe.score(X_val, y_val)

print(f"Validation accuracy: {val_score:.3f}")


Validation accuracy: 0.997


## Create Candidate Feature

In [ ]:
def add_candidate_column(df: pd.DataFrame, config: dict) -> pd.DataFrame:
    df = df.copy()

    bandgap_col   = config["filters"]["bandgap_column"]
    sem_min       = config["filters"]["semiconductor_min"]
    sem_max       = config["filters"]["semiconductor_max"]
    trans_min     = config["filters"]["transparent_min"]
    toxic_elements = config["filters"]["toxic_elements"]

    df[bandgap_col] = pd.to_numeric(df[bandgap_col], errors="coerce")
    in_semiconductor_range = df[bandgap_col].between(sem_min, sem_max)
    is_transparent = df[bandgap_col] > trans_min

    if "ehull" in df.columns:
        df["ehull"] = pd.to_numeric(df["ehull"], errors="coerce")
        is_stable = df["ehull"] < 0.1
    else:
        is_stable = True

    if "formula" in df.columns:
        tokens = df["formula"].fillna("").astype(str).str.findall(r"[A-Z][a-z]?")
        has_toxic = tokens.apply(lambda t: any(el in t for el in toxic_elements))
    else:
        has_toxic = False

    df["is_candidate"] = (
        in_semiconductor_range &
        is_transparent &
        is_stable &
        (has_toxic == False)
    ).astype(int)

    return df


In [39]:
config = {
    "filters": {
        "bandgap_column": "optb88vdw_bandgap",
        "semiconductor_min": 0.5,
        "semiconductor_max": 5.0,
        "transparent_min": 2.5,
        "toxic_elements": ["Pb", "Cd", "As", "Hg"]
    }
}

df_feat = add_candidate_column(df, config)
df_feat = featurizer.apply_plan(df_feat)

print(df_feat[["optb88vdw_bandgap", "ehull", "id_candidate"]].head())


/home/brent/src/ml-final-project-team2/code/jarvis/featurizer.py:13: FutureWarning:

Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`



   optb88vdw_bandgap   ehull  id_candidate
0              0.000  0.0423             0
1              0.000  0.0456             0
2              0.000  0.3183             0
3              0.472  0.0000             0
4              0.000  0.0156             0


In [40]:
df.head()

,jid,spg_number,spg_symbol,formula,formation_energy_peratom,func,optb88vdw_bandgap,atoms,slme,magmom_oszicar,spillage,elastic_tensor,effective_masses_300K,kpoint_length_unit,maxdiff_mesh,maxdiff_bz,encut,optb88vdw_total_energy,epsx,epsy,epsz,mepsx,mepsy,mepsz,modes,magmom_outcar,max_efg,avg_elec_mass,avg_hole_mass,icsd,dfpt_piezo_max_eij,dfpt_piezo_max_dij,dfpt_piezo_max_dielectric,dfpt_piezo_max_dielectric_electronic,dfpt_piezo_max_dielectric_ionic,max_ir_mode,min_ir_mode,n-Seebeck,p-Seebeck,n-powerfact,p-powerfact,ncond,pcond,nkappa,pkappa,ehull,Tc_supercon,dimensionality,efg,xml_data_link,typ,exfoliation_energy,spg,crys,density,poisson,raw_files,nat,bulk_modulus_kv,shear_modulus_gv,mbj_bandgap,hse_gap,reference,search
0,JVASP-90856,129,P4/nmm,TiCuSiAs,-0.42762,OptB88vdW,0.000,"{'lattice_mat': [[3.566933224304235, 0.0, -0.0], [0.0, 3.566933224304235, -0.0], [-0.0, -0.0, 9.397075454186664]], 'coords': [[2.6751975000000003, 2.6751975000000003, 7.376101754328542], [0.8917325, 0.8917325, 2.0209782456714573], [0.8917325, 2.6751975000000003, 4.69854], [2.6751975000000003, 0.8917325, 4.69854], [0.8917325, 2.6751975000000003, 0.0], [2.6751975000000003, 0.8917325, 0.0], [2.6751975000000003, 2.6751975000000003, 2.8894795605846353], [0.8917325, 0.8917325, 6.507600439415366]], 'elements': ['Ti', 'Ti', 'Cu', 'Cu', 'Si', 'Si', 'As', 'As'], 'abc': [3.56693, 3.56693, 9.39708], 'angles': [90.0, 90.0, 90.0], 'cartesian': True, 'props': ['', '', '', '', '', '', '', '']}",na,0.0,na,na,"{'p': 'na', 'n': 'na'}",60,na,na,650,-3.37474,76.23,76.23,54.0402,na,na,na,na,0.0,na,na,na,,na,na,na,na,na,na,na,na,na,na,na,na,na,na,na,0.0423,na,3D-bulk,[],<a href=https://www.ctcms.nist.gov/~knc6/static/JARVIS-DFT/JVASP-90856.xml target='_blank' >JVASP-90856</a>,bulk,na,129,tetragonal,5.956,na,[],8,na,na,na,na,mp-1080455,-As-Cu-Si-Ti
1,JVASP-86097,221,Pm-3m,DyB6,-0.41596,OptB88vdW,0.000,"{'lattice_mat': [[4.089078911208881, 0.0, 0.0], [-0.0, 4.089078911208881, -0.0], [0.0, -0.0, 4.089078911208881]], 'coords': [[0.0, 0.0, 0.0], [0.8121488741123271, 2.04454, 2.04454], [2.04454, 3.276931125887674, 2.04454], [2.04454, 0.8121488741123271, 2.04454], [3.276931125887674, 2.04454, 2.04454], [2.04454, 2.04454, 0.8121488741123271], [2.04454, 2.04454, 3.276931125887674]], 'elements': ['Dy', 'B', 'B', 'B', 'B', 'B', 'B'], 'abc': [4.08908, 4.08908, 4.08908], 'angles': [90.0, 90.0, 90.0], 'cartesian': True, 'props': ['', '', '', '', '', '', '']}",na,0.0,na,na,"{'p': 'na', 'n': 'na'}",65,na,na,1000,-5.79186,140.9742,140.9742,140.9742,na,na,na,na,0.0,na,na,na,"613601,613595,150623",na,na,na,na,na,na,na,na,na,na,na,na,na,na,na,0.0456,na,3D-bulk,[],<a href=https://www.ctcms.nist.gov/~knc6/static/JARVIS-DFT/JVASP-86097.xml target='_blank' >JVASP-86097</a>,bulk,na,221,cubic,5.522,na,"[OPT-LOPTICS,JVASP-86097.zip,https://ndownloader.figshare.com/files/25258037, OPT-Bandst,JVASP-86097.zip,https://ndownloader.figshare.com/files/24011081, DFT-SCF,JVASP-86097.zip,https://ndownloader.figshare.com/files/23566679]",7,na,na,na,na,mp-568319,-B-Dy
2,JVASP-64906,119,I-4m2,Be2OsRu,0.04847,OptB88vdW,0.000,"{'lattice_mat': [[-1.833590720595598, 1.833590720595598, 3.4849681632445244], [1.833590720595598, -1.833590720595598, 3.4849681632445244], [1.833590720595598, 1.833590720595598, -3.4849681632445244]], 'coords': [[0.0, 0.0, 0.0], [1.83359, 0.0, 1.742485], [0.0, 0.0, 3.48497], [0.0, 1.83359, 1.742485]], 'elements': ['Be', 'Be', 'Os', 'Ru'], 'abc': [4.34386, 4.34386, 4.34386], 'angles': [130.0642, 130.0642, 73.3042], 'cartesian': True, 'props': ['', '', '', '']}",na,0.0,na,na,"{'p': 'na', 'n': 'na'}",60,na,na,1100,-4.65924,215.4488,215.4488,159.4368,na,na,na,na,0.0,23.94,na,na,,na,na,na,na,na,na,na,na,na,na,na,na,na,na,na,0.3183,na,intercalated ion,"[[Be, a, -1.504, 0.001, 0.001, 0.001, -1.504, 0.001, 0.001, 0.001, 3.009], [Be, d, -0.62, 0.001, 0.001, 0.001, -0.62, 0.001, 0.001, 0.001, 1.239], [Os, b, 11.97, -0.0, 0.0, -0.0, 11.97, 0.0, 0.0, 0.0, -23.94], [Ru, c, 6.036, 0.0, 0.0, 0.0, 6.036, 0.0